# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI(# 中转Key或者直连Key
    api_key='sk-EMA9xdeuT3krvtvE558e351d65704aB9A28b350b5dC6A78b',         
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1')

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条位于郁郁葱葱的草地上的木栈道。栈道直通向远方，引人遐想。四周长满了绿草和不同高度的灌木，远处可以看到一些树木。天空广阔，蓝天中散布着几朵白云，为整个场景增添了和平与宁静的感觉。图片的色彩饱和度高，呈现出一种生机勃勃和清新自然的氛围。整体上，这是一幅展现自然美景和夏日氛围的图像。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种'

### 封装成一个函数 query_image_description

In [9]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(# 中转Key或者直连Key
    api_key='sk-EMA9xdeuT3krvtvE558e351d65704aB9A28b350b5dC6A78b',         
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1')  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [11]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url,prompt='解释一下这张图，并解释文本')
print(content)

这张图是一张幽默图片，展示了两只不同状态的狗，进行对比，以此来表达当人们年龄相差并不大时因为生活方式和习惯不同而可能导致的巨大差异。左边的狗被描绘得像是一个肌肉非常发达的人型，看起来非常强壮和壮观。右边的狗则显得普通，稍显肥胖和慵懒。

图片上方的文本“16岁的狗”和“1代后勤狗”可能意指这是同一时间出生的狗，但生活环境和照顾方式不同造成了它们巨大的身体差异。左边的狗下方的文字解释了它的生活方式：“我周遭一片光明，意志坚定，若米饭荒哪，人力车的力大属我，旗帜的风向归我。” 这段话似乎在描述这只


In [4]:
pip install markdown

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/105.4 kB ? eta -:--:--
     ---------------------- ---------------- 61.4/105.4 kB 1.7 MB/s eta 0:00:01
     ------------------------------------ - 102.4/105.4 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 105.4/105.4 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [11]:
from openai import OpenAI
import base64
import requests
import json
from IPython.display import display, Markdown

client = OpenAI(# 中转Key或者直连Key
    api_key='sk-EMA9xdeuT3krvtvE558e351d65704aB9A28b350b5dC6A78b',         
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1')  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容?", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return display(Markdown(content))
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [9]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

图中展示了1980年到2020年间美国、中国、日本和德国的国内生产总值（GDP）比较。横轴表示年份，纵轴表示GDP（以万亿美元计）。不同颜色的线代表不同国家：

- **蓝色线**：美国
- **红色线**：中国
- **紫色线**：日本
- **绿色线**：德国

从图中可以看出以下几点：

1. **美国**的GDP持续领先，且呈稳定增长态势。
2. **中国**的GDP从1990年代后期开始显著增长，特别是进入2000年后增长更加迅速，接近美国的GDP水平。
3. **日本**的GDP在1990年代初期达到高峰后，波动较大，但总体趋于平稳。
4. **德国**的GDP增长较为缓慢，整体上呈轻微上升趋势。

这张图很好地反映了过去四十年这四个国家经济增长的轨迹和比较。

```markdown
以下为GDP数据的简化版表格：

| 年份 | 美国(GDP万亿美元) | 中国(GDP万亿美元) | 日本(GDP万亿美元) | 德国(GDP万亿美元) |
|------|-----------------|-----------------|-----------------|-----------------|
| 1980 | 2.86            | 0.31            | 1.11            | 0.95            |
| 1990 | 5.98            | 0.36            | 3.12            | 1.71            |
| 2000 | 10.25           | 1.21            | 4.87            | 1.94            |
| 2010 | 14.96           | 5.11            | 5.70            | 3.42            |
| 2020 | 21.43           | 14.34           | 5.08            | 3.84            |

数据源自图表展示。
```

None


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [20]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片是一张笔记本的照片，内容主要介绍了几种针对深度学习模型的参数调整技术，特别是在使用transformer架构的NLP模型中。笔记内容如下：

1. **Prompt Tuning（简模型调整）** - 这是一种减少模型参数修改的技术，通过在模型的输入部分（通常是embedding层）添加一些特定的“提示”（提示是一组特定词汇或语句），从而引导模型生成特定的输出。这里有两种提示的示例，第一种是简单的连续输入X，第二种则是前缀调整，通过改变W值来影响变换输出Y。

2. **LoRA（Low-Rank Adaptation, 低秩适应）** - 提到了一种新的参数适应方法，通过对模型中的权重矩阵W进行小规模的修改（ΔW），来调整模型的行为，而不需要重新训练整个模型。这种技术可以通过矩阵分解来实现ΔW的计算，以减少所需的额外参数数量，从而减少内存使用和计算成本。

3. **内存使用对比** - 笔记最后提到了使用LoRA技术相比标凈LoRA技术的内存消耗比较，例如LLaMa-653B模型使用LoRA需要78GB, 而QLoRA只需要48GB。

这张笔记非常适合对深度学习，尤其是NLP领域中的模型调整方法有兴趣的读者。它涉及了一些高级的概念和技术，可用于优化模型的性能和效率。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [6]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这是一张笔记本的照片，记录了关于机器学习、尤其是自然语言处理（NLP）中的不同技术和方法的笔记。这些笔记包括了多种模型调整方法和参考信息。

### 左侧部分内容概览：
- **半监督学习** 和 **弱监督学习** 的提及。
- 提到了 **转换器（Transformers）** 模型。
- **PEFT 和 SOTA （State of the Art）** - 提及了性能基准和方法。
- **PEFT 方法细节**
  - **Prompt Tuning**: 模型微调技术，用于调整模型以更好地应对特定任务。
  - **Adapter（2019, Google）**, **Prefix（2021, Stanford）**, **Prompt（2021, Google）** 等具体技术。
  - 提到了多个基于指令的调整技术和具体引用，如 **LAIAN（2021.09）**,
    **BLOOM**, **ChatGLM**, **Alpaca**.

### 右侧部分内容概览：
- 讨论了 **多模态说明特征抽取（multi-modality instruction feature extraction）** 和 **LLMs（Large Language Models）**.
- **New** 方法中包括 **LORA**, **GLORA**, **AdapterA**, **PETC (2022)** 和 **Ig3**.
- **Refine in LLMs**:
  - 涉及到改良LLM的具体技术，如通过项目加强（Project down）和积极提升（Project up）。
- **MAM Adapters**:
  - **Scaled parallel adapter**, **IFN adapter** 等方法，强调通过结构改良提升模型性能。

这些笔记显示了一个正在学习或研究机器学习和自然语言处理领域的革新技术的人。注意到，该笔记包含多个先进的模型微调和性能提升技术，适合用于深层次的技术研究或实际应用优化。

![](./images/handwriting_1.jpg)

In [7]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片显示的是一个笔记本上用手写记录的内容，主要涉及机器学习和自然语言处理（NLP）中的一些技术和方法。内容主要用中文和英文混合编写，看起来像是在做学术研究或学习的笔记。

### 左侧内容包括：
1. **模型类别和评测**
   - 数据模型：提到了`Transformer`模型及其相关评测（Benchmark）和方法（如PEFT，PBFT，响应式FT等）。
   - **PEFT** 与 **SOTA (State of the Art)** 和 **PBFT Methods**。

2. **PEFT 细分介绍**
   - 涉及到Prompt Tuning（提示调整），包括Adapter（2019, Google）、Prefix（2021, Stanford）、及其他相关技术如Prompt（2021, Google）、P-Tuning V1/V2等。
   - 提及了**指令调优**（Instruction tuning）、**FT LLMs**（Fine-tuning Large Language Models）等。

3. **其他技术和模型**
   - 提到了一些具体的模型如**Liang et al., 2019**、**GPT**, **XLHF**, **LLM**, **BLOOM**, **Alpaca**, **MOSS**。

### 右侧内容包括：
1. **多模态及新技术**
   - 讨论到**multi-modality** Instruction FT LLMs，如 **LLaMA (13B)**。
   - 新技术如**LoRA**，**GLoRA**，**Adalora**。

2. **技术的扩展与应用**
   - **Prefix-tuning & Adapters**的进一步应用和技术细分。

通过以上笔记内容，可以看出此人可能在研究或学习有关自然语言处理的高级技术和方法，特别是与**语言模型微调**和**适应器模式**相关的内容。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。